In [1]:
import torch
import nibabel as nib
import niwidgets as nw
import pathlib
from data_utils import MRAData
from network import R2AttUNet
import numpy as np
import os
import random

import warnings
warnings.filterwarnings(action='ignore')

%load_ext autoreload
%autoreload 2

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,3"

model = R2AttUNet(in_channels=1, out_channels=2)

model_path = pathlib.Path("/home/dhaval/pytorch_3dunet/r2attunet20.pth")
model.load_state_dict(torch.load(str(model_path)))
model.cuda()
print("Loaded model")

Loaded model


In [3]:
adam_path = pathlib.Path("/home/dhaval/adam_data/")

data = MRAData(adam_path, mode="val")

patient_num = random.randint(0, int(len(data)))

MRA_path = data.cases_dirs[patient_num]
print(str(MRA_path).split('/')[-1])
raw_path = str(MRA_path.joinpath("pre/TOF.nii.gz"))
seg_path = str(MRA_path.joinpath("aneurysms.nii.gz"))

# raw_affine = nib.load(raw_path).affine
seg_affine = nib.load(seg_path).affine

location_path = MRA_path.joinpath("location.txt")

# Get center of aneurysm from location.txt file of case
aneurysm_location_coords = data.get_aneurysm_coords(location_path)

print("Location: ", aneurysm_location_coords)

raw_widget = nw.NiftiWidget(raw_path)
raw_widget.nifti_plotter()

seg_widget = nw.NiftiWidget(seg_path)
seg_widget.nifti_plotter()

10056F
Location:  [[261.0, 235.0, 59.0, 2.26], [281.0, 271.0, 68.0, 2.72]]


<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=279, continuous_update=False, description='x', max=559), IntSlider(value…

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=279, continuous_update=False, description='x', max=559), IntSlider(value…

In [4]:
import misc as ms

save_name = MRA_path.joinpath("output.nii.gz")

x = nib.load(raw_path)
x = np.asarray(x.dataobj).astype(np.int32)
x = torch.from_numpy(x).to(dtype=torch.float)
x = x.unsqueeze(0).cuda().detach()

seg_proxy = nib.load(seg_path)
seg_data = np.asarray(seg_proxy.dataobj).astype(np.int32)
target = torch.from_numpy(seg_data).to(dtype=torch.float)

output = ms.test(model, x, patch_size=44, stride=42)
output = output.detach()

0 / 392
1 / 392
2 / 392
3 / 392
4 / 392
5 / 392
6 / 392
7 / 392
8 / 392
9 / 392
10 / 392
11 / 392
12 / 392
13 / 392
14 / 392
15 / 392
16 / 392
17 / 392
18 / 392
19 / 392
20 / 392
21 / 392
22 / 392
23 / 392
24 / 392
25 / 392
26 / 392
27 / 392
28 / 392
29 / 392
30 / 392
31 / 392
32 / 392
33 / 392
34 / 392
35 / 392
36 / 392
37 / 392
38 / 392
39 / 392
40 / 392
41 / 392
42 / 392
43 / 392
44 / 392
45 / 392
46 / 392
47 / 392
48 / 392
49 / 392
50 / 392
51 / 392
52 / 392
53 / 392
54 / 392
55 / 392
56 / 392
57 / 392
58 / 392
59 / 392
60 / 392
61 / 392
62 / 392
63 / 392
64 / 392
65 / 392
66 / 392
67 / 392
68 / 392
69 / 392
70 / 392
71 / 392
72 / 392
73 / 392
74 / 392
75 / 392
76 / 392
77 / 392
78 / 392
79 / 392
80 / 392
81 / 392
82 / 392
83 / 392
84 / 392
85 / 392
86 / 392
87 / 392
88 / 392
89 / 392
90 / 392
91 / 392
92 / 392
93 / 392
94 / 392
95 / 392
96 / 392
97 / 392
98 / 392
99 / 392
100 / 392
101 / 392
102 / 392
103 / 392
104 / 392
105 / 392
106 / 392
107 / 392
108 / 392
109 / 392
110 / 392


In [5]:
target = target.unsqueeze(0)

print("Check dice")
dice = ms.dice_coeff(output, target)
print("Dice: ", dice.item())

Check dice
Dice:  0.5016181468963623


In [6]:
output = output.detach().cpu().numpy()

out_img = nib.Nifti1Image(output, seg_affine)
nib.save(out_img, str(save_name))

test_widget = nw.NiftiWidget(save_name)
test_widget.nifti_plotter()

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=279, continuous_update=False, description='x', max=559), IntSlider(value…